In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import ast

from sklearn import preprocessing
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer, r2_score
from sklearn.svm import SVR

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
bert_with_ner = pd.read_csv('/content/drive/My Drive/TCC/csv/final/bert_with_ner.csv').fillna(0).replace('\n','', regex=True).drop(columns='Unnamed: 0')
bert_without_ner = pd.read_csv('/content/drive/My Drive/TCC/csv/final/bert_without_ner.csv').fillna(0).replace('\n','', regex=True).drop(columns='Unnamed: 0')
vectorized_with_ner = pd.read_csv('/content/drive/My Drive/TCC/csv/final/vectorized_with_ner.csv').fillna(0).replace('\n','', regex=True).drop(columns='Unnamed: 0')
vectorized_without_ner = pd.read_csv('/content/drive/My Drive/TCC/csv/final/vectorized_without_ner.csv').fillna(0).replace('\n','', regex=True).drop(columns='Unnamed: 0')

In [ ]:
vectorized_without_ner.head()

,notas,DET,NOUN,ADP,PROPN,VERB,ADJ,PUNCT,CCONJ,PRON,SPACE,ADV,SCONJ,AUX,NUM,SYM,X,INTJ,vectorized_content,vectorized_excerpt
0,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,10.119796,14.894006
1,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,7.907671,18.088019
2,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,8.283214,9.208959
3,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,9.333219,21.802164
4,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,10.603291,8.455762


In [ ]:
# bert_with_ner['vectorized_content_bert'] = bert_with_ner['vectorized_content_bert'].apply(ast.literal_eval)

In [ ]:
# bert_with_ner.head(2)
# bert_with_ner['vectorized_content_bert'].head()

In [4]:
# Conjunto de treino e teste do bert com named entity
bert_with_ner_train = bert_with_ner.sample(frac=0.8)
bert_with_ner_test = bert_with_ner.drop(bert_with_ner_train.index)

# Conjunto de treino e teste do bert sem named entity
bert_without_ner_train = bert_without_ner.sample(frac=0.8)
bert_without_ner_test = bert_without_ner.drop(bert_without_ner_train.index)

# Conjunto de treino e teste da vetorização com named entity
vectorized_with_ner_train = vectorized_with_ner.sample(frac=0.8)
vectorized_with_ner_test = vectorized_with_ner.drop(vectorized_with_ner_train.index)

# Conjunto de treino e teste da vetorização sem named entity
vectorized_without_ner_train = vectorized_without_ner.sample(frac=0.8)
vectorized_without_ner_test = vectorized_without_ner.drop(vectorized_without_ner_train.index)




In [5]:
# Variaveis para treinamento do dataset vetorizado sem ner
X_vec = vectorized_without_ner_train.drop(columns=['notas'])
y_vec = vectorized_without_ner_train['notas']

# Variaveis para treinamento do dataset vetorizado com ner
X_vec_ner = vectorized_with_ner_train.drop(columns=['notas'])
y_vec_ner = vectorized_with_ner_train['notas']

# Variaveis para treinamento do dataset vetorizado com BERT sem ner
X_bert = bert_without_ner_train.drop(columns=['notas'])
y_bert = bert_without_ner_train['notas']

# Variaveis para treinamento do dataset vetorizado com BERT com ner
X_bert_ner = bert_with_ner_train.drop(columns=['notas'])
y_bert_ner = bert_with_ner_train['notas']


In [56]:
# Variaveis para teste do dataset vetorizado sem ner
x_vec_test = vectorized_without_ner_test.drop(columns=['notas'])
y_vec_test = vectorized_without_ner_test['notas']

# Variaveis para teste do dataset vetorizado com ner
X_vec_ner_test = vectorized_with_ner_test.drop(columns=['notas'])
y_vec_ner_test = vectorized_with_ner_test['notas']

# Variaveis para teste do dataset vetorizado com BERT sem ner
X_bert_test = bert_without_ner_test.drop(columns=['notas'])
y_bert_test = bert_without_ner_test['notas']

# Variaveis para teste do dataset vetorizado com BERT com ner
X_bert_ner_test = bert_with_ner_test.drop(columns=['notas'])
y_bert_ner_test = bert_with_ner_test['notas']


In [10]:
grid_vec = GridSearchCV(estimator=LinearRegression(), param_grid={}, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
grid_vec.fit(X_vec,y_vec)

GridSearchCV(cv=10, estimator=LinearRegression(), n_jobs=-1, param_grid={},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [11]:
grid_vec.predict(x_vec_test)

array([6.67759512, 7.06032032, 6.65914933, 6.92122528, 6.72146723,
       6.82913334, 6.93525106, 5.60926801, 6.9586482 , 7.05950949,
       6.13916119, 7.06822751, 7.06498536, 6.78080522, 5.99529823,
       6.45203923, 6.63815231, 5.98333897, 6.21189136, 7.14582484,
       7.12309097, 7.06747681, 6.58528271, 7.16975348, 6.4465125 ,
       7.07945845, 6.77424024, 6.70566143, 6.99390944, 6.99307795,
       7.07427193, 6.73936951, 6.75601256, 6.9891288 , 7.07657848,
       6.39663282, 6.30944426, 6.95348206, 5.19803421, 7.07951302,
       6.95094377, 6.94106144, 6.57948851, 6.64455943, 7.01149773,
       7.00311871, 7.00727461, 6.97823203, 6.07973074, 7.01942441,
       6.66423656, 6.95993987, 7.09114138, 7.04401452, 6.97423959,
       6.90866978, 6.59077322, 7.00770056, 6.77567522, 6.60811048,
       7.02186774, 6.70410747, 6.76981921, 6.73367673, 6.60784427,
       6.68171383, 7.06351963, 6.21371519, 6.95153394, 6.91672484,
       6.63717309, 6.94771145, 7.04339377, 7.06409316, 6.67579

In [12]:
grid_vec.score(x_vec_test,y_vec_test)

-0.030833535615805818

In [ ]:


grid_vec_ner = GridSearchCV(estimator=LinearRegression(), param_grid={}, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
grid_vec_ner.fit(X_vec_ner,y_vec_ner)

GridSearchCV(cv=10, estimator=LinearRegression(), n_jobs=-1, param_grid={},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [ ]:
grid_vec_ner.predict(X_vec_ner)[:30]

array([6.38772894, 6.83359717, 6.52120464, 6.93539873, 6.60328025,
       6.28147485, 7.04814563, 6.88579904, 6.70222419, 7.7057483 ,
       6.94017032, 6.43100421, 6.68990774, 6.68552861, 6.21025187,
       8.26866634, 9.17816228, 7.07690678, 6.71806443, 6.91540939,
       6.60595904, 6.94647189, 7.19161632, 6.41981976, 6.90040485,
       6.50261863, 6.67268862, 7.45575949, 6.59178291, 6.2106917 ])

In [ ]:
grid_vec_ner.score(X_vec_ner,y_vec_ner)

0.014179135846510427

In [ ]:
# grid_vec_ner.cv_results_

In [ ]:
grid_bert = GridSearchCV(estimator=LinearRegression(), param_grid={}, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
grid_bert.fit(X_bert,y_bert)

GridSearchCV(cv=10, estimator=LinearRegression(), n_jobs=-1, param_grid={},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [ ]:
grid_bert.predict(X_bert_test)

array([6.88964614, 6.9096508 , 7.00304435, 6.38036277, 6.81525871,
       6.09344532, 5.88232615, 7.34833257, 6.41894157, 6.7139239 ,
       6.42900176, 6.41708099, 6.1947585 , 7.35214382, 7.37141035,
       7.31718504, 6.69027352, 6.94340561, 6.69496672, 7.23794099,
       6.54148616, 6.43583393, 6.72443849, 6.54647329, 6.98550677,
       6.64477814, 6.68673866, 6.57100349, 6.68310293, 6.52616279,
       7.81109269, 6.94033709, 6.55030361, 7.25198776, 7.124489  ,
       6.27897093, 6.5295721 , 6.76489429, 7.95350802, 6.18512236,
       6.33412811, 6.01484642, 6.29654742, 7.41447747, 6.86317326,
       6.85103059, 6.60774683, 6.61871049, 6.87518744, 6.35476043,
       6.81554102, 7.08453185, 6.24101627, 6.279634  , 7.27248114,
       6.83592404, 6.65200212, 7.20519286, 8.29460757, 7.17816691,
       6.36513393, 7.30952595, 6.61362511, 6.64952706, 6.91513558,
       6.81470664, 8.02893472, 6.69493766, 6.40234484, 6.34924429,
       6.26447286, 6.00906737, 6.79903159, 6.85652312, 7.82156

In [ ]:
grid_bert.score(X_bert_test,y_bert_test)

0.0219341060100815

In [ ]:
# grid_bert.cv_results_

In [ ]:
grid_bert_ner = GridSearchCV(estimator=LinearRegression(), param_grid={}, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
grid_bert_ner.fit(X_bert_ner,y_bert_ner)

GridSearchCV(cv=10, estimator=LinearRegression(), n_jobs=-1, param_grid={},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [ ]:
grid_bert_ner.predict(X_bert_ner_test)

array([6.45270461, 6.85820081, 7.0870556 , 6.16464955, 6.19436441,
       5.96957938, 6.05066091, 6.87140818, 6.09324123, 6.42279895,
       6.96533646, 7.77622452, 7.15004444, 6.91202693, 7.61470877,
       7.23613683, 5.86644651, 6.29623102, 5.69045015, 5.79552982,
       7.60040455, 6.593825  , 7.04845542, 7.24576473, 7.77637866,
       6.62268107, 7.08870045, 7.064512  , 6.61518756, 6.71373993,
       7.19612978, 6.6183069 , 6.66603059, 6.9580674 , 6.27126672,
       8.0319681 , 6.03010888, 6.99694556, 6.21706685, 8.4993651 ,
       5.75810026, 6.16233116, 6.33033243, 6.6267869 , 6.86267404,
       6.85312475, 6.90494552, 6.42497444, 6.68436765, 8.43368594,
       7.5581996 , 7.28039878, 6.9712677 , 6.16873732, 6.62714262,
       6.91516642, 6.72154672, 6.82825694, 6.20048083, 7.08388943,
       8.35139006, 6.47329754, 6.23986839, 6.53483322, 6.72966403,
       6.16354779, 6.88775726, 7.68765479, 6.2392199 , 5.79872255,
       7.22258171, 6.98881556, 6.14003115, 7.13206916, 6.82200

In [ ]:
grid_bert_ner.score(X_bert_ner_test,y_bert_ner_test)

-0.0428436876497964

In [ ]:
grid_bert_ner.cv_results_

{'mean_fit_time': array([0.00393541]),
 'std_fit_time': array([0.00041756]),
 'mean_score_time': array([0.00250292]),
 'std_score_time': array([0.00026843]),
 'params': [{}],
 'split0_test_r2-score': array([-0.03931249]),
 'split1_test_r2-score': array([-0.0802361]),
 'split2_test_r2-score': array([0.06478831]),
 'split3_test_r2-score': array([-0.09106722]),
 'split4_test_r2-score': array([-0.15222868]),
 'split5_test_r2-score': array([0.01079658]),
 'split6_test_r2-score': array([0.01873522]),
 'split7_test_r2-score': array([0.01817071]),
 'split8_test_r2-score': array([-0.00619779]),
 'split9_test_r2-score': array([0.03353418]),
 'mean_test_r2-score': array([-0.02230173]),
 'std_test_r2-score': array([0.06374533]),
 'rank_test_r2-score': array([1], dtype=int32),
 'split0_train_r2-score': array([0.02662447]),
 'split1_train_r2-score': array([0.04115908]),
 'split2_train_r2-score': array([0.02495212]),
 'split3_train_r2-score': array([0.03462393]),
 'split4_train_r2-score': array([0.03

In [ ]:
# parameters = {'C': [1, 10], 
#           'gamma': [0.001, 0.01, 1]}
# model = SVC()
# grid = GridSearchCV(estimator=model, param_grid=parameters)
# grid.fit(vectors, newsgroups_train.target)
# print(grid)
# # summarize the results of the grid search
# print(grid.best_score_)
# print(grid.best_estimator_)

In [47]:
parameters = {'C': [1, 10], 'gamma': [0.001,0.01,1]}
model = SVR()
svr_grid_vec = GridSearchCV(estimator=model, param_grid=parameters, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
svr_grid_vec.fit(X_vec, y_vec)

GridSearchCV(cv=10, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': [0.001, 0.01, 1]},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [ ]:
svr_grid_vec_result =svr_grid_vec.cv_results_
# svr_grid_vec_result

In [48]:
svr_grid_vec.best_params_

{'C': 10, 'gamma': 0.01}

In [49]:
svr_grid_vec.score(x_vec_test, y_vec_test)

-0.15800312550942452

In [52]:
svr_grid_vec.predict(x_vec_test)

array([ 9.18190126,  9.09232081,  9.0736039 ,  8.91585262,  8.93269745,
        9.01314884,  9.08850335, -0.14745632,  8.96560318,  9.08542476,
        7.73947037,  9.10974345,  9.06313794,  8.46422796,  2.9303455 ,
        8.09847243,  9.15693052,  9.90053056,  9.52002962,  9.10462187,
        9.0112429 ,  8.99549433,  7.94375466,  9.06960079,  7.52876912,
        9.11173259,  9.12390772,  8.95344947,  9.02441736,  9.03304311,
        9.00211539,  8.94488204,  8.70652393,  9.0034667 ,  9.07446009,
        6.58658466,  6.61708705,  9.05745652,  2.59095958,  9.05051397,
        8.95066827,  9.06211142,  9.00762352,  8.92357731,  9.09647217,
        9.08270016,  9.04727443,  8.99212323,  9.73619541,  8.96634918,
        8.23973214,  8.94399707,  9.0492785 ,  9.06649876,  8.9114192 ,
        9.08663707,  7.74455978,  9.09523867,  8.40194909,  8.76125833,
        9.02904909,  8.15066356,  9.08729027,  9.15835546,  7.782101  ,
        8.05010641,  9.00498009,  9.49041719,  8.98734604,  9.05

In [58]:
svr_grid_vec_ner = GridSearchCV(estimator=model, param_grid=parameters, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
svr_grid_vec_ner.fit(X_vec_ner, y_vec_ner)

GridSearchCV(cv=10, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': [0.001, 0.01, 1]},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [26]:
svr_grid_vec_ner_result = svr_grid_vec_ner.cv_results_
# svr_grid_vec_ner_result

In [23]:
svr_grid_vec_ner.best_params_

{'C': 10, 'gamma': 0.01}

In [59]:
svr_grid_vec_ner.score(X_vec_ner_test, y_vec_ner_test)

-0.21739924469913308

In [60]:
svr_grid_vec_ner.predict(X_vec_ner_test)

array([ 8.97996205,  9.07176484,  8.96646606,  3.56326295,  8.85812288,
        8.91571507,  3.26266501,  8.45305796,  4.43089626,  9.19134766,
        9.19122036,  8.95938731,  9.00758964, 10.13573596,  9.07391702,
        9.17526212,  9.34186704,  9.21221399,  9.86896315,  7.41524401,
        9.10289997,  9.01788683,  8.94564844,  8.94277502,  9.35172207,
        9.81570431,  9.48398211,  9.50021449,  9.03624102,  8.39758157,
        7.0242834 ,  8.99217247,  9.04830812,  9.003746  ,  9.31880527,
        9.08171224,  6.46517822,  9.10223166,  9.04401484,  9.03864564,
        8.89458015,  9.08925031,  8.62714248,  8.16468387,  9.19162701,
        9.04489563,  9.0335712 ,  9.03022815,  9.08587111,  9.07017419,
        8.96446864,  9.00672258,  9.09583092,  9.74852155,  9.20952596,
        9.34493808,  9.05688847,  9.06649549,  9.0564666 ,  9.05007794,
        9.00396574,  9.00307297,  9.20069099,  9.23247537,  7.52715854,
        7.55170608,  3.73058164,  8.92917104,  9.74164661,  9.06

In [62]:
svr_grid_bert = GridSearchCV(estimator=model, param_grid=parameters, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
svr_grid_bert.fit(X_bert, y_bert)

GridSearchCV(cv=10, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': [0.001, 0.01, 1]},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [30]:
svr_grid_bert.best_params_

{'C': 10, 'gamma': 0.001}

In [36]:
svr_grid_bert_result = svr_grid_bert.cv_results_
# svr_grid_bert_result

In [64]:
svr_grid_bert.score(X_bert_test, y_bert_test)

0.00040962482789863586

In [65]:
svr_grid_bert.predict(X_bert_test)

array([6.79870587, 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.90009894,
       6.7987047 , 6.7987047 , 6.79874284, 6.79870474, 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.79870504, 6.79907923,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.79870478, 7.48919622, 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.81870046,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 , 6.7987047 ,
       6.7987016 , 6.7987047 , 6.7987047 , 6.7987047 , 6.79870

In [37]:
svr_grid_bert_ner = GridSearchCV(estimator=model, param_grid=parameters, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=10,n_jobs=-1)
svr_grid_bert_ner.fit(X_bert_ner, y_bert_ner)

GridSearchCV(cv=10, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': [0.001, 0.01, 1]},
             refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [38]:
svr_grid_bert_ner.best_params_

{'C': 10, 'gamma': 0.001}

In [66]:
svr_grid_bert_ner_result = svr_grid_bert_ner.cv_results_
# svr_grid_bert_ner_result

In [67]:
svr_grid_bert_ner.score(X_bert_ner_test, y_bert_ner_test)

-0.0008314237564910343

In [68]:
svr_grid_bert_ner.predict(X_bert_ner_test)

array([6.80117314, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80120942, 6.80117198, 6.80117198, 6.80117231,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117361, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117198, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117209, 6.80117198, 6.80117198, 6.80117198, 6.80117198,
       6.80117196, 6.80117198, 6.80117198, 6.80117198, 6.80116

In [14]:
# i = 0
# while(i < 100000000000):
#   i+=1
#   print(i)